In [101]:
using CSV, DataFrames, Gadfly, GLM, Distributions

In [102]:
train = CSV.read("train.csv")
size(train,1)

14094

In [103]:
Y₁ = collect(train[:,:NA_Sales])
Y₂ = collect(train[:,:Global_Sales])
X₁ = collect(train[:,:Critic_Score])
X₂ = collect(train[:,:Critic_Count])
X₃ = collect(train[:,:User_Score])
X₄ = collect(train[:,:User_Count])
X₅ = collect(train[:,:Other_Sales])
X₆ = collect(train[:,:JP_Sales])
X₇ = collect(train[:,:Rating])
X₈ = collect(train[:,:Genre])

n = size(Y₁,1)
Xtotal = hcat(ones(n),X₅,X₆)

14094×3 Array{Union{Missing, Float64},2}:
 1.0  0.0100241    0.0505242  
 1.0  0.00179053   0.0392108  
 1.0  0.00179738   0.000374026
 1.0  0.14001      0.00138205 
 1.0  0.00953506   0.0694222  
 1.0  0.0302705    0.14039    
 1.0  0.000962246  0.161401   
 1.0  0.011419     0.00168636 
 1.0  0.000182071  0.0314134  
 1.0  0.0517094    0.000293506
 1.0  0.690459     0.0615083  
 1.0  0.00124541   0.00158265 
 1.0  0.0412362    0.00153098 
 ⋮                            
 1.0  0.00137629   0.000796422
 1.0  0.0614496    0.000601123
 1.0  0.00882679   0.000202291
 1.0  0.000427063  0.000767142
 1.0  3.93275e-5   0.0019588  
 1.0  0.0014472    0.031158   
 1.0  0.000931152  0.0206156  
 1.0  0.00902387   0.000716744
 1.0  0.000880418  0.000754146
 1.0  0.0018516    0.00179397 
 1.0  0.00193711   0.000435447
 1.0  0.00120583   0.000879716

In [104]:

##Critic score, User Score, Critic count, User Count 
moyenneCriticCount = floor(mean(collect(skipmissing(train[:,:Critic_Score]))))
moyenneCriticScore =floor(mean(collect(skipmissing(train[:,:Critic_Count]))))
moyenneUser_Score = floor(mean(collect(skipmissing(train[:,:User_Score]))))
moyenneUser_Count = floor(mean(collect(skipmissing(train[:,:User_Count]))))


for i in eachindex(X₁) 
    if ismissing(X₁[i]) || ismissing(X₂[i]) || ismissing(X₃[i]) || ismissing(X₄[i])
        X₁[i]= moyenneCriticCount
        X₂[i]= moyenneCriticScore
        X₃[i]= moyenneUser_Score
        X₄[i]= moyenneUser_Count
    end
end
Xtotal = hcat(Xtotal,X₁,X₂,X₃,X₄)

14094×7 Array{Union{Missing, Float64},2}:
 1.0  0.0100241    0.0505242    74.0  40.0  7.5    13.0
 1.0  0.00179053   0.0392108    69.0  26.0  7.0   157.0
 1.0  0.00179738   0.000374026  69.0  26.0  7.0   157.0
 1.0  0.14001      0.00138205   69.0  26.0  7.0   157.0
 1.0  0.00953506   0.0694222    67.0  19.0  7.5    61.0
 1.0  0.0302705    0.14039      72.0  33.0  8.2   179.0
 1.0  0.000962246  0.161401     69.0  26.0  7.0   157.0
 1.0  0.011419     0.00168636   63.0  31.0  6.6     7.0
 1.0  0.000182071  0.0314134    69.0  26.0  7.0   157.0
 1.0  0.0517094    0.000293506  69.0  26.0  7.0   157.0
 1.0  0.690459     0.0615083    84.0  57.0  7.4  2142.0
 1.0  0.00124541   0.00158265   69.0  26.0  7.0   157.0
 1.0  0.0412362    0.00153098   69.0  26.0  7.0   157.0
 ⋮                                          ⋮          
 1.0  0.00137629   0.000796422  69.0  26.0  7.0   157.0
 1.0  0.0614496    0.000601123  72.0   8.0  4.6    32.0
 1.0  0.00882679   0.000202291  69.0  26.0  7.0   157.0
 1.0  

In [105]:
#Rating
for i in eachindex(X₇) 
    if ismissing(X₇[i])
        X₇[i] = "missing"
    end
end

ratingSet = Set(X₇) 
ratings = collect(ratingSet) 

## Comme on sait que les meilleurs variables explicatives sont le rating
## E, M, missing et K-A, on retire tous les autres
filter!(x->x≠"EC",ratings)
filter!(x->x≠"E10+",ratings)
filter!(x->x≠"RP",ratings)
filter!(x->x≠"T",ratings)
filter!(x->x≠"AO",ratings)
##Contient toutes les variables qualitatives du rating
# Xrating = Array{Int64}[14094]


for i= 1:(length(ratings)-1)
    categorie = ratings[i]
    qualitative = zeros(n)
    
    ## on parcourt tous les element de la variable Rating
    for j in eachindex(X₇) 
        if X₇[j] == categorie
            qualitative[j] = 1
        end
    end
    
    Xtotal = hcat(Xtotal,qualitative)
end

Xtotal


14094×10 Array{Union{Missing, Float64},2}:
 1.0  0.0100241    0.0505242    74.0  40.0  7.5    13.0  1.0  0.0  0.0
 1.0  0.00179053   0.0392108    69.0  26.0  7.0   157.0  0.0  0.0  1.0
 1.0  0.00179738   0.000374026  69.0  26.0  7.0   157.0  1.0  0.0  0.0
 1.0  0.14001      0.00138205   69.0  26.0  7.0   157.0  1.0  0.0  0.0
 1.0  0.00953506   0.0694222    67.0  19.0  7.5    61.0  0.0  1.0  0.0
 1.0  0.0302705    0.14039      72.0  33.0  8.2   179.0  0.0  1.0  0.0
 1.0  0.000962246  0.161401     69.0  26.0  7.0   157.0  0.0  0.0  1.0
 1.0  0.011419     0.00168636   63.0  31.0  6.6     7.0  0.0  0.0  0.0
 1.0  0.000182071  0.0314134    69.0  26.0  7.0   157.0  0.0  0.0  1.0
 1.0  0.0517094    0.000293506  69.0  26.0  7.0   157.0  0.0  0.0  1.0
 1.0  0.690459     0.0615083    84.0  57.0  7.4  2142.0  0.0  1.0  0.0
 1.0  0.00124541   0.00158265   69.0  26.0  7.0   157.0  1.0  0.0  0.0
 1.0  0.0412362    0.00153098   69.0  26.0  7.0   157.0  0.0  0.0  1.0
 ⋮                                

In [106]:
# ## Genre


# for i in eachindex(X₈) 
#     if ismissing(X₈[i])
#         X₈[i] = "missing"
#     end
# end

# genreSet = Set(X₈) 
# genres = collect(genreSet) 

# print(length(genres))
# ##Contient toutes les variables qualitatives de genre
# # Xgenre = Array{Float64}[]


# for i= 1:(length(genres)-1)
#     genre = genres[i]
#     qualitative = zeros(n)
    
#     ## on parcourt tous les element de la variable Rating
#     for j in eachindex(X₈) 
#         if X₈[j] == genre
#             qualitative[j] = 1
#         end
#     end
    
#     Xtotal = hcat(Xtotal,qualitative)
# end

# # Xtotal = hcat(Xtotal,Xgenre)

In [107]:
p = size(Xtotal,2)-1
Xtotal

14094×10 Array{Union{Missing, Float64},2}:
 1.0  0.0100241    0.0505242    74.0  40.0  7.5    13.0  1.0  0.0  0.0
 1.0  0.00179053   0.0392108    69.0  26.0  7.0   157.0  0.0  0.0  1.0
 1.0  0.00179738   0.000374026  69.0  26.0  7.0   157.0  1.0  0.0  0.0
 1.0  0.14001      0.00138205   69.0  26.0  7.0   157.0  1.0  0.0  0.0
 1.0  0.00953506   0.0694222    67.0  19.0  7.5    61.0  0.0  1.0  0.0
 1.0  0.0302705    0.14039      72.0  33.0  8.2   179.0  0.0  1.0  0.0
 1.0  0.000962246  0.161401     69.0  26.0  7.0   157.0  0.0  0.0  1.0
 1.0  0.011419     0.00168636   63.0  31.0  6.6     7.0  0.0  0.0  0.0
 1.0  0.000182071  0.0314134    69.0  26.0  7.0   157.0  0.0  0.0  1.0
 1.0  0.0517094    0.000293506  69.0  26.0  7.0   157.0  0.0  0.0  1.0
 1.0  0.690459     0.0615083    84.0  57.0  7.4  2142.0  0.0  1.0  0.0
 1.0  0.00124541   0.00158265   69.0  26.0  7.0   157.0  1.0  0.0  0.0
 1.0  0.0412362    0.00153098   69.0  26.0  7.0   157.0  0.0  0.0  1.0
 ⋮                                

In [108]:
β̂₁ = (Xtotal'Xtotal)\Xtotal'Y₁

β̂₂ = (Xtotal'Xtotal)\Xtotal'Y₂

10-element Array{Float64,1}:
 -0.2300277916066933    
  4.8354377773571535    
  2.242312041425307     
  0.006243405862818217  
  0.005114951055507173  
 -0.030853428434262337  
  0.00014321335730794826
  0.09965441723405721   
  0.03427738635844263   
 -0.10544306846557087   

In [109]:
test = CSV.read("test.csv")
head(test)

,ID,Platform,Year_of_Release,Genre,Publisher,JP_Sales,Other_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
,Int64⍰,String⍰,Int64⍰,String⍰,String⍰,Float64⍰,Float64⍰,Int64⍰,Int64⍰,Float64⍰,Int64⍰,String⍰,String⍰
1,14095,GBA,2002,Fighting,Acclaim Entertainment,0.000520983,0.00025736,24,11,3.7,10,Powerhead Games,E
2,14096,PS3,2008,Action,Codemasters,0.000966418,0.0204632,72,35,7.7,27,4J Studios,T
3,14097,N64,1997,Sports,Konami Digital Entertainment,0.259834,0.0489383,missing,missing,missing,missing,missing,missing
4,14098,GC,2002,Fighting,Midway Games,0.00088793,0.00826891,81,20,8.7,43,Midway,M
5,14099,X360,2010,Role-Playing,Sega,0.000439246,0.0388025,63,68,7.2,201,Obsidian Entertainment,M
6,14100,DS,2009,Adventure,Ubisoft,0.001625,0.00016853,missing,missing,7.5,4,Longtail Studios,T


In [110]:
T₁ = collect(test[:,:Critic_Score])
T₂ = collect(test[:,:Critic_Count])
T₃ = collect(test[:,:User_Score])
T₄ = collect(test[:,:User_Count])

T₅ = collect(test[:,:Other_Sales])
T₆ = collect(test[:,:JP_Sales])

T₇ = collect(test[:,:Rating])

T₈ = collect(test[:,:Genre])

k = length(T₁)
Ttotal = hcat(ones(k),T₅,T₆)



2490×3 Array{Union{Missing, Float64},2}:
 1.0  0.00025736   0.000520983
 1.0  0.0204632    0.000966418
 1.0  0.0489383    0.259834   
 1.0  0.00826891   0.00088793 
 1.0  0.0388025    0.000439246
 1.0  0.00016853   0.001625   
 1.0  0.00805139   1.11841    
 1.0  0.0309118    0.000595432
 1.0  0.00120239   0.0413245  
 1.0  0.0514995    0.00108301 
 1.0  0.0819643    0.11108    
 1.0  0.000175463  0.000982736
 1.0  0.00153822   0.001875   
 ⋮                            
 1.0  0.210153     0.000980181
 1.0  0.0100436    0.000601615
 1.0  0.00827426   0.00026977 
 1.0  0.729499     1.10951    
 1.0  0.00165461   0.0214866  
 1.0  0.0113141    0.00130528 
 1.0  0.0117503    0.000249642
 1.0  0.000353334  0.0083478  
 1.0  0.030626     0.000289506
 1.0  0.0183706    0.0690372  
 1.0  0.00808898   0.00120171 
 1.0  0.000559854  0.190844   

In [111]:

##Critic score, User Score, Critic count, User Count pour les donnees de test
for i in eachindex(T₁) 
    if ismissing(T₁[i]) || ismissing(T₂[i]) || ismissing(T₃[i]) || ismissing(T₄[i])
        T₁[i]= moyenneCriticCount
        T₂[i]= moyenneCriticScore
        T₃[i]= moyenneUser_Score
        T₄[i]= moyenneUser_Count
    end
end
Ttotal = hcat(Ttotal,T₁,T₂,T₃,T₄)

2490×7 Array{Union{Missing, Float64},2}:
 1.0  0.00025736   0.000520983  24.0  11.0  3.7   10.0
 1.0  0.0204632    0.000966418  72.0  35.0  7.7   27.0
 1.0  0.0489383    0.259834     69.0  26.0  7.0  157.0
 1.0  0.00826891   0.00088793   81.0  20.0  8.7   43.0
 1.0  0.0388025    0.000439246  63.0  68.0  7.2  201.0
 1.0  0.00016853   0.001625     69.0  26.0  7.0  157.0
 1.0  0.00805139   1.11841      59.0  20.0  7.7    6.0
 1.0  0.0309118    0.000595432  76.0  44.0  8.6   14.0
 1.0  0.00120239   0.0413245    69.0  26.0  7.0  157.0
 1.0  0.0514995    0.00108301   69.0  26.0  7.0  157.0
 1.0  0.0819643    0.11108      93.0  45.0  9.1  135.0
 1.0  0.000175463  0.000982736  80.0  10.0  7.5    4.0
 1.0  0.00153822   0.001875     73.0  27.0  7.7   20.0
 ⋮                                          ⋮         
 1.0  0.210153     0.000980181  86.0  37.0  6.7  228.0
 1.0  0.0100436    0.000601615  69.0  26.0  7.0  157.0
 1.0  0.00827426   0.00026977   47.0  11.0  7.0    5.0
 1.0  0.729499     1.109

In [112]:
#Rating
for i in eachindex(T₇) 
    if ismissing(T₇[i])
        T₇[i] = "missing"
    end
end

for i= 1:(length(ratings)-1)
    categorie = ratings[i]
    qualitative = zeros(k)
    
    ## on parcourt tous les element de la variable Rating
    for j in eachindex(T₇) 
        if T₇[j] == categorie
            qualitative[j] = 1
        end
    end
    
    Ttotal = hcat(Ttotal,qualitative)
end


In [114]:
NA_Sales = Ttotal*β̂₁

Global_Sales = Ttotal*β̂₂

predictions = DataFrame(ID = test[:ID], NA_Sales = NA_Sales, Global_Sales = Global_Sales)
CSV.write("predictions.csv",predictions)

"predictions.csv"